In [63]:
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re,string,unicodedata
from nltk import pos_tag
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression,SGDClassifier

In [39]:
df = pd.read_csv('Musical_instruments_reviews.csv')
df.head()
df.reviewText.fillna("",inplace = True)
del df['reviewerID']
del df['asin']
del df['reviewerName']
del df['helpful']
del df['unixReviewTime']
del df['reviewTime']

df['text'] = df['reviewText'] + ' ' +  df['summary']
del df['reviewText']
del df['summary']

def is_good_or_bad(rating):
    if (rating <= 3):
        return 0
    else:
        return 1

df.overall = df.overall.apply(is_good_or_bad)
df.text[0]

"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing, good"

In [46]:

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
    
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            pos = pos_tag([i.strip()])
            word = lemmatizer.lemmatize(i.strip(),get_simple_pos(pos[0][1]))
            final_text.append(word.lower())
    return " ".join(final_text)

df.text = df.text.apply(lemmatize_words)

In [67]:
X_train,X_test,y_train,y_test = train_test_split(df.text,df.overall,test_size = 0.2 , random_state = 1)


In [68]:
#word2vec
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(X_train)
#transformed test reviews
cv_test_reviews=cv.transform(X_test)



In [70]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=0)

lr_bow=lr.fit(cv_train_reviews,y_train)
print(lr_bow)

print(f'Model test accuracy: {lr.score(cv_test_reviews, y_test)*100:.3f}%')


LogisticRegression(C=1, max_iter=500, random_state=0)
Model test accuracy: 87.628%
